# Regresion Logistica (RegLog)

## Caso: Diabetes

Vamos a utilizar un dataset que contiene datos de pacientes mujeres, llamado ***diabetes.csv***. El objetivo del dataset es predecir de forma diagnóstica si un paciente tiene diabetes o no, basándose en ciertas mediciones de diagnóstico.

Los datos del dataset son:

*   Pregnancies: Cantidad de embarazos
*   Glucose: Concentración de glucosa en plasma a 2 horas, en prueba oral de tolerancia
*   BloodPressure: Presión arterial diastólica (mm Hg)
*   SkinThickness: Espesor del pliegue de la piel del tríceps (mm)
*   Insulin:  Niveles de insulina en suero (micro U / ml)
*   BMI: Índice de masa corporal (peso en kg / altura en m^2)
*   DiabetesPedigreeFunction: Función pedigree de diabetes (representa la probabilidad de que contraiga la enfermedad extrapolando la historia de sus antepasados)
*   Age: Edad de la paciente (en años)
*   Outcome: Es el atributo clase a predecir, donde "1" indica con diabetes y "0" sin diabetes

## A. Análisis Exploratorio y Descriptivo


In [ ]:
# Importamos las librerías que necesitamos

# Instalamos skopt para hacer busqueda bayesiana de hiperparametros
!pip install scikit-optimize
from skopt import BayesSearchCV
from sklearn import metrics

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import pydotplus

# Matriz de confusión
from sklearn.metrics import confusion_matrix

# Cargamos el dataset de diabetes
df_diabetes = pd.read_csv('https://raw.githubusercontent.com/unlam-fcdin/UNLaM_FCDIN/master/diabetes.csv', sep = ',')
df_diabetes.head(5)


Veamos los tipos de datos del dataset.

In [ ]:
df_diabetes.dtypes

Veamos el tamaño del dataframe

In [ ]:
print("Tamaño del dataframe : {}".format(df_diabetes.shape))

¿Cómo es la distribución de la variable clase Outcome?

*Outcome* es la variable clase que vamos a predecir, la cual indica si el paciente es diabético o no. El **valor 1** significa que la persona es diabética mientas que el **valor 0** significa que no lo es.

In [ ]:
sns.countplot(x='Outcome', data=df_diabetes)

De acuerdo al gráfico vemos que hay más pacientes que no tienen diabetes, pero ¿cuántos exactamente?

In [ ]:
df_diabetes.groupby('Outcome').size()

In [ ]:
268 / (500 + 268)

Podemos identificar que de las 768 personas, 500 no son diabéticas (valor 0) y 268 sí lo son (valor 1).


Analicemos cómo afecta la posibilidad o no de tener diabetes, según los siguientes aspectos:

*   Por la cantidad de embarazos

In [ ]:
# Creamos un gráfico boxplot para analizar la CANTIDAD DE EMBARAZOS de la paciente respecto a la CLASE
plt.figure(figsize=(4, 8))
s=sns.boxplot(x="Outcome", y="Pregnancies", data=df_diabetes)
s.plot()

¿Se observa alguna relación respecto a la cantidad de embarazos entre las pacientes que tienen o no diabetes?


*   Por edad de la paciente

In [ ]:
plt.figure(figsize=(4, 8))
s=sns.boxplot(x="Outcome", y="Age", data=df_diabetes)
s.plot()

¿Se observa alguna relación respecto a la edad entre las pacientes que tienen o no diabetes?

¿Cómo es la distribución de las edades de las pacientes?

In [ ]:
# Construimos un gráfico de densidad
plt.title('Age')
sns.kdeplot(df_diabetes['Age'], shade=False) # shade indica si el gráfico es sombreado o no

In [ ]:
# Veamos la media en valor absoluto
print("Media: {}".format(df_diabetes['Age'].mean()))
print("Mediana: {}".format(df_diabetes['Age'].median()))

*   Por el índice de masa corporal (BMI)


Tenemos los valores de referencia para entender la distribución de la variable:
> ![Referencia](http://www.annacara.com/fotostratamientos/39foto01m.jpg)

In [ ]:
plt.figure(figsize=(4, 6))
s=sns.boxplot(x="Outcome", y="BMI", data=df_diabetes)
s.plot()

¿Se observa alguna relación respecto al índice de masa corporal (BMI) de las pacientes que tienen o no diabetes?

## B. Preparación y Transformación de Datos

#### **Valores Faltantes**

Comprobamos si alguno de los campos del dataset contiene valores faltantes o nulos.

In [ ]:
val_nulos = df_diabetes.isnull().sum()
print(val_nulos)

#### **Valores Erróneos**

Analicemos la información de los principales indicadores estadísticos del dataset:

In [ ]:
df_diabetes.describe()

Veamos gráficamente los atributos que tienen valores en cero (0).

In [ ]:
atributos_ceros = (df_diabetes == 0).sum(axis=0)
atributos_ceros = pd.DataFrame(atributos_ceros, columns=['Cantidad de Ceros'])
atributos_ceros = atributos_ceros.sort_values(by=['Cantidad de Ceros'], ascending=True)
atributos_ceros.drop(['Outcome'], inplace = True)
atributos_ceros.plot(kind='barh', figsize=(15,5), color='orange', grid=False)

**Pregunta**: ¿Pueden tener valor cero (0) los siguientes atributos?

* Glucose
* BloodPressure
* SkinThickness
* Insulin
* BMI

En estas columnas, un valor en 0 no tiene sentido y, por lo tanto, indicaría un valor erróneo.

Veamos cuántos valores así tiene cada atributo por cada etiqueta del atributo clase?

---



In [ ]:
# Atributo Glucose
print("Valores en cero: ", df_diabetes[df_diabetes.Glucose == 0].shape[0])
df_diabetes[df_diabetes.Glucose == 0].groupby('Outcome').size()

In [ ]:
# Atributo BloodPressure
print("Valores en cero: ", df_diabetes[df_diabetes.BloodPressure == 0].shape[0])
df_diabetes[df_diabetes.BloodPressure == 0].groupby('Outcome').size()

In [ ]:
# Atributo SkinThickness
print("Valores en cero: ", df_diabetes[df_diabetes.SkinThickness == 0].shape[0])
df_diabetes[df_diabetes.SkinThickness == 0].groupby('Outcome').size()

In [ ]:
# Atributo Insulin
print("Valores en cero: ", df_diabetes[df_diabetes.Insulin == 0].shape[0])
df_diabetes[df_diabetes.Insulin == 0].groupby('Outcome').size()

In [ ]:
# Atributo BMI
print("Valores en cero: ", df_diabetes[df_diabetes.BMI == 0].shape[0])
df_diabetes[df_diabetes.BMI == 0].groupby('Outcome').size()

Hay varias formas de manejar valores erróneos:

* **Eliminarlos**: esto no es posible en la mayoría de los casos porque significaría perder información valiosa. En el caso de los atributos "SkinThickness" e "Insulin" significan perder muchos datos. Pero podría funcionar para los atributos "Glucose", "BloodPressure" y "BMI".

* **Imputarlos**: esto podría funcionar para algunos datos, por ejemplo usando la *media* o *mediana* de los valores. Tener cuidado de no generar valores muy distorsionados de los reales.

* **Crear una nueva variable dicotómica iferror**: si se va a optar por imputar los valores o eliminar la columna, se puede crear una variable dictómica que analice si la variable original era correcta o no y ver como se comporta en el modelo.

* **No usar los atributos**: si el atributo contiene muchos valores erróneos, podríamos evitar usarlos y ver cómo se comporta el modelo. No es posible saberlo de antemano.

Veamos cómo es la distribución de los atributos con problemas para ver cuál estrategia elegir.

In [ ]:
# Construimos un histograma múltiple
f, axes = plt.subplots(2, 3, figsize=(15, 10), sharex=False)

sns.distplot(df_diabetes["Glucose"], kde = False, color = "skyblue",ax=axes[0, 0]) # axes [fila,columna] indica la posición del histograma
sns.distplot(df_diabetes["BloodPressure"],kde = False, color = "olive", ax=axes[0, 1])
sns.distplot(df_diabetes["SkinThickness"],kde = False, color = "red", ax=axes[0, 2])
sns.distplot(df_diabetes["Insulin"],kde = False, color = "gold", ax=axes[1, 0])
sns.distplot(df_diabetes["BMI"],kde = False, color = "teal", ax=axes[1, 1])

sns.distplot(df_diabetes["Pregnancies"],kde = False, color = "orange", ax=axes[1, 2])

#### **Valores Outliers**

Analicemos la información de cada atributo para analizar posibles outliers.

- **Pregnancies**



In [ ]:
# Creamos un gráfico boxplot para analizar la variable "PREGNANCIES"
plt.figure(figsize=(10, 3))
sns.boxplot(x="Pregnancies", data=df_diabetes, orient = 'h', palette="Set2")

In [ ]:
# Distribución de la variable "PREGNANCIES"
print(df_diabetes['Pregnancies'].median())
print(df_diabetes['Pregnancies'].std())
sns.kdeplot(df_diabetes['Pregnancies'], shade=True)

In [ ]:
#print(df_diabetes['Pregnancies'].value_counts())

# Obtenemos el valor límite superior para el outlier
print(df_diabetes['Pregnancies'].mean())
print(df_diabetes['Pregnancies'].std())
outlier_superior = df_diabetes['Pregnancies'].mean() + 3*df_diabetes['Pregnancies'].std()   # +/-3 desvíos estándar es valor outlier
print('Valor Outlier Superior: {:.2f}'.format(outlier_superior))

- **Glucose**

In [ ]:
# Creamos un gráfico boxplot para analizar la variable "GLUCOSE"
plt.figure(figsize=(10, 3))
sns.boxplot(x="Glucose", data=df_diabetes, orient = 'h', palette="Set2")

Observamos que este atributo no tiene valores outliers.

- **BloodPressure**

In [ ]:
# Creamos un gráfico boxplot para analizar la variable "BLOODPRESSURE"
plt.figure(figsize=(10, 3))
sns.boxplot(x="BloodPressure", data=df_diabetes, orient = 'h', palette="Set2")

In [ ]:
#print(df_diabetes['BloodPressure'].value_counts())

# Obtenemos el valor límite para el outlier
outlier_inferior = df_diabetes['BloodPressure'].mean() - 1*df_diabetes['BloodPressure'].std()   # -2.5 desvíos estándar es valor outlier inferior
outlier_superior = df_diabetes['BloodPressure'].mean() + 1*df_diabetes['BloodPressure'].std()   # +2.5 desvíos estándar es valor outlier superior
print('Valor Outlier Inferior: {:.2f} - Valor Outlier Superior {:.2f}'.format(outlier_inferior, outlier_superior))

In [ ]:
# Distribución de la variable "BloodPressure"
sns.kdeplot(df_diabetes['BloodPressure'], shade=True)

Del gráfico bloxpot vemos que el **límite inferior atípico es menor a 40**. El cálculo obtenido del outlier inferior aproximadamente el mismo, pero dejaría algunas observaciones con valores outlier. Por lo tanto, tomamos como límite inferior para la imputación el valor 40.

- **SkinThickness**

In [ ]:
# Creamos un gráfico boxplot para analizar la variable "SKINTHICKNESS"
plt.figure(figsize=(10, 3))
sns.boxplot(x="SkinThickness", data=df_diabetes, orient = 'h', palette="Set2")

In [ ]:
#print(df_diabetes['SkinThickness'].value_counts())

# Obtenemos el valor límite para el outlier
outlier_inferior = df_diabetes['SkinThickness'].mean() - 1.5*df_diabetes['SkinThickness'].std()   # -1.5 desvíos estándar es valor outlier inferior
outlier_superior = df_diabetes['SkinThickness'].mean() + 1.5*df_diabetes['SkinThickness'].std()   # +1.5 desvíos estándar es valor outlier superior
print('Valor Outlier Inferior: {:.2f} - Valor Outlier Superior {:.2f}'.format(outlier_inferior, outlier_superior))

In [ ]:
# Distribución de la variable "SkinThickness"
sns.kdeplot(df_diabetes['SkinThickness'], shade=True)

- **Insulin**

In [ ]:
# Creamos un gráfico boxplot para analizar la variable "INSULINE"
plt.figure(figsize=(10, 3))
sns.boxplot(x="Insulin", data=df_diabetes, orient = 'h', palette="Set2")

In [ ]:
# Distribución de la variable "Insulin"
sns.kdeplot(df_diabetes['Insulin'], shade=True)

Dado que hay observaciones con niveles de insulina por encima de los 250 micro U / ml, no vamos a considerarlas outliers, ya que pueden ser características significantes en pacientes con diabetes.

- **BMI**

In [ ]:
# Creamos un gráfico boxplot para analizar la variable "BMI"
plt.figure(figsize=(10, 3))
sns.boxplot(x="BMI", data=df_diabetes, orient = 'h', palette="Set2")

In [ ]:
#print(df_diabetes['BMI'].value_counts())

# Obtenemos el valor límite para el outlier
outlier_superior = df_diabetes['BMI'].mean() + 2.5*df_diabetes['BMI'].std()   # +2.5 desvíos estándar es valor outlier superior
print('Valor Outlier Superior {:.2f}'.format(outlier_superior))

In [ ]:
# Distribución de la variable "BMI"
sns.kdeplot(df_diabetes['BMI'], shade=True)

- **DiabetesPedigreeFunction**

In [ ]:
# Creamos un gráfico boxplot para analizar la variable "DiabetesPedigreeFunction"
plt.figure(figsize=(10, 3))
sns.boxplot(x="DiabetesPedigreeFunction", data=df_diabetes, orient = 'h', palette="Set2")

In [ ]:
#print(df_diabetes['DiabetesPedigreeFunction'].value_counts())

# Obtenemos el valor límite para el outlier
outlier_superior = df_diabetes['DiabetesPedigreeFunction'].mean() + 2*df_diabetes['DiabetesPedigreeFunction'].std()   # +2 desvíos estándar es valor outlier superior
print('Valor Outlier Superior {:.2f}'.format(outlier_superior))

In [ ]:
# Distribución de la variable "DiabetesPedigreeFunction"
sns.kdeplot(df_diabetes['DiabetesPedigreeFunction'], shade=True)

- **Age**

In [ ]:
# Creamos un gráfico boxplot para analizar la variable "AGE"
plt.figure(figsize=(10, 3))
sns.boxplot(x="Age", data=df_diabetes, orient = 'h', palette="Set2")

#### **Análisis de correlación de variables**

Una matriz de correlación permite estudiar la relación lineal o comportamiento que puede existir entre dos o más variables.

  - Correlación positiva: ocurre cuando una variable aumenta y la otra también.
  - Correlación negativa: es cuando una variable aumenta y la otra disminuye.
  - Sin correlación: no hay una relación aparente entre las variables.

Calculamos la matriz de correlacion entre todas las variables del dataset

In [ ]:
# Matriz de correlación
df_diabetes.corr()

La misma matriz podemos visualizarla mediante un **mapa de calor**.

In [ ]:
plt.figure(figsize=(8,8))
sns.heatmap(df_diabetes.corr(), annot=True, vmax=.7, cmap ='Blues')

Seleccionamos sólo la correlacion de la variable objetivo "*Outcome*".

In [ ]:
df_diabetes_corr = df_diabetes.corr()[["Outcome"]]*100
df_diabetes_corr

Borramos la correlación de la variable objetivo consigo misma y ordenamos las variables predictoras de acuerdo a la correlación.

In [ ]:
df_diabetes_corr = df_diabetes_corr.drop("Outcome", axis=0)
df_diabetes_corr = df_diabetes_corr.sort_values(["Outcome"], ascending=False) # ordenamos en forma descendente
df_diabetes_corr = abs(df_diabetes_corr)
df_diabetes_corr

Podemos visualizar lo mismo mediante un mapa de calor.

In [ ]:
plt.figure(figsize=(8, 8))
sns.heatmap(df_diabetes_corr, robust=True, linewidths=.5, annot=True, )

#### *Análisis preliminar*

Del mapa de calor podemos observar que existe una correlación más alta entre las variables *Glucose*, *BMI*, *Age*, *Pregnancies* e *Insuline*  respecto a la posibilidad de que una paciente tenga diabetes.

#### **Estandarización**

Como los atributos tienen unidades y magnitudes diferentes, los normalizamos para mejorar la predicción. Aprovechando la estandarización, separamos del dataset las variables predictoras (X) y la variable clase (y).

In [ ]:
# Aplicamos la estandarización
from sklearn.preprocessing import StandardScaler

scaler_X = StandardScaler(with_mean=True, with_std=True)
scaler_X.fit(df_diabetes.drop(["Outcome"], axis = 1)) # entrenamos los valores quitandole la variable clase
X_diabetes = pd.DataFrame(scaler_X.transform(df_diabetes.drop(["Outcome"],axis = 1),),
                          columns = ['Pregnancies','Glucose','BloodPressure','SkinThickness', 'Insulin',
                                     'BMI','DiabetesPedigreeFunction','Age'])  # aplicamos la transformacion
X_diabetes.head()

**VEN ALGO INCORRECTO EN LA CELDA ANTERIOR???? Sino lo ven, vuelvan a verla.**
<br>
<br>
Separamos ahora la variable clase.

In [ ]:
y_diabetes = df_diabetes["Outcome"]
y_diabetes.head()

## D. Partición del conjunto de datos

Dividimos el dataset original en dos conjuntos de datos:

*  Conjunto de entrenamiento (70%)
*  Conjunto de prueba (30%)

In [ ]:
# Importamos la librería que necesitamos
from sklearn.model_selection import train_test_split

# Dividimos X e y con train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_diabetes, y_diabetes, test_size=0.3, stratify = y_diabetes, random_state=42)

## E. Construcción del modelo con Regresión Logística

En esta parte aprenderemos cómo aplicar un modelo de Regresión Logística en problemas de clasificación.

#### Referencia: [Documentación de Regresión Logística](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)

#### **Aplicación del algoritmo**

Como vimos, el algoritmo de regresión logística es apropiado para realizar predicciones cuando la variable clase es binaria. Cuando la variable a predecir es "multi clase", el algoritmo puede usar un esquema "uno vs el resto" para mantener una relación binaria.

Con los datos ya particionados, procedemos a entrenar el modelo y a evaluar los resultados utilizando el conjunto de prueba.

In [ ]:
# Importamos la librería que necesitamos
from sklearn.linear_model import LogisticRegression # regresión logística para clasificación
from sklearn.metrics import accuracy_score # métrica de evaluación

RegLog = LogisticRegression()
RegLog.fit(X_train,y_train) # Creamos y entrenamos el clasificador de regresión logística

Veamos qué peso tiene cada variable en la regresión.
Este análisis lo realizamos a través de los coeficientes asignados en la recta de regresión a cada variable, cuanto mayor el coeficiente, mayor el peso en la recta.
Esto es similar al "feature importance" en los árboles de decisión.

In [ ]:
pd.DataFrame({'Atributo':X_train.columns,
              'importancia':abs(RegLog.coef_[0])}).sort_values('importancia', ascending=False).head(10)

Podemos ver que las variables con mayor correlación numérica, también salieron importantes en la regresión, pero no en el mismo orden.
¿Por qué? ¿Qué pasa si quitamos alguna variable importante y recalculamos el modelo? ¿Mejora o empeora?

In [ ]:
X_train.describe()

#### **Evaluación del modelo y Matriz de Confusión**

Con el modelo ya entrenado, utilizamos el *conjunto de prueba* para verificar su capacidad de predicción.

In [ ]:
# Calculamos y mostramos la matriz de confusión del modelo
y_pred_RLog = RegLog.predict(X_test)
confusion_matrix(y_test, y_pred_RLog)
pd.crosstab(y_test, y_pred_RLog, rownames=['Real'], colnames=['Predicho'])

Veamos cuál fue la **exactitud** (Accuracy) obtenida por el modelo.

In [ ]:
#Exactitud del modelo
print('Exactitud (accuracy) del modelo: {:.2f} %'.format(accuracy_score(y_test, y_pred_RLog)*100))
print("-"*100)

# Reporte del clasificador
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_RLog))

####**Optimización con BayesSearchCV**

Nuevamente utilizamos la técnica de **validación cruzada** o **cross-validation** para encontrar el mejor modelo de clasificación.
El algoritmo de búsqueda Bayesiana es una mejora sobre los metodos standards GridSearch (probar todo) y RandomSearch (prueba aleatoria).
<br><br>
En este caso el algoritmo hace una busqueda de los posibles hot-zones donde la metrica definida (en este caso AUC score) "crece" y trata de hacer una búsqueda enfocada en mejorar el score y descartar pruebas innecesarias hasta un punto de convergencia y/o superar el maximo de iteraciones definidos `n_iter`.

In [ ]:
parametros = {'penalty': ['l2'],
               'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}

# Realizamos la búsqueda con BayesSearchCV
modeloRL = LogisticRegression()
gs = BayesSearchCV(modeloRL, parametros, scoring='precision', verbose=0 , n_jobs=-1, cv = 3, random_state=3, n_iter=50)
gs.fit(X_train, y_train)

Mostramos los mejores resultados obtenidos a partir de los hiper-parámetros utilizados.

In [ ]:
print("Exactitud (Accuracy) dentro del CV: {:.2f}%".format(gs.best_score_ * 100))
print("Parámetros del estimador: " + str(gs.best_estimator_))

In [ ]:
# Predigo los valores
y_pred_RLog = gs.best_estimator_.predict(X_test)

#Exactitud del modelo
print('Exactitud (accuracy) del modelo: {:.2f} %'.format(accuracy_score(y_test, y_pred_RLog)*100))
print("-"*100)

# Reporte del clasificador
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred_RLog))

## F. *Otros enfoques: ¿Qué pasaría si no escalamos/imputamos el dataset?*

##### Volvemos a cargar el set de datos, lo normalizamos y dividimos X e y (entrenamiento y prueba)

In [ ]:
# Antes volvemos a cargar el set de datos con una versión sin imputar, para comparar los resultados de ambas
df_diabetes_si = pd.read_csv('https://raw.githubusercontent.com/unlam-fcdin/UNLaM_FCDIN/master/diabetes.csv', sep = ',')

#### **Opción 1**: Escalando las variables

In [ ]:
# Escalamos las variables
scaler_X = StandardScaler(with_mean=True, with_std=True)
scaler_X.fit(df_diabetes_si.drop(["Outcome"],axis = 1)) # entrenamos los valores quitandole la variable clase
X_diabetes_si = pd.DataFrame(scaler_X.transform(df_diabetes_si.drop(["Outcome"],axis = 1),),
                          columns = ['Pregnancies','Glucose','BloodPressure','SkinThickness', 'Insulin',
                                     'BMI','DiabetesPedigreeFunction','Age'])  # aplicamos la transformacion
y_diabetes_si = df_diabetes_si["Outcome"]

# Dividimos en entrenamiento y prueba
X_train_si, X_test_si, y_train_si, y_test_si = train_test_split(X_diabetes_si, y_diabetes_si, test_size=0.3, stratify = y_diabetes_si, random_state=42)

X_train_si.head()

Probamos el **modelo KNN** (el mejor hasta ahora) sin imputar outliers ni valores erróneos

In [ ]:
parametros = {'penalty': ['l2'],
               'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}

# Realizamos la búsqueda con Bayes Search
modeloRL = LogisticRegression()
gs = BayesSearchCV(modeloRL, parametros, scoring='roc_auc', verbose=0 , n_jobs=-1, cv = 3, random_state=3, n_iter=50)
gs.fit(X_train_si, y_train_si)

# Mostramos los resultados del mejor modelo
print("-"*100)
print("AUC en CV: {:.2f}%".format(gs.best_score_ * 100))
print("-"*100)
print("Parámetros del estimador: " + str(gs.best_estimator_))
print("-"*100)

# Predecimos los casos de test para ver el accuracy
y_pred_si = gs.best_estimator_.predict(X_test_si)

Veamos los resultados

In [ ]:
#Exactitud del modelo
print('AUC del modelo: {:.2f} %'.format(metrics.roc_auc_score(y_test_si, y_pred_si)*100))
print("-"*100)

# Reporte del clasificador
from sklearn.metrics import classification_report
print(classification_report(y_test_si, y_pred_si))

#### **Opción 2**: Sin escalar las variables

In [ ]:
# Sin escalar las variables
X_diabetes_si_se = df_diabetes_si.drop(["Outcome"],axis = 1)
y_diabetes_si_se = df_diabetes_si["Outcome"]

# Dividimos en entrenamiento y prueba
X_train_si_se, X_test_si_se, y_train_si_se, y_test_si_se = train_test_split(X_diabetes_si_se, y_diabetes_si_se, test_size=0.3, stratify = y_diabetes_si_se, random_state=42)

X_train_si_se.head()

Ahora probamos el **modelo KNN** sin estandarizar los valores.

In [ ]:
parametros = {'penalty': ['l2'],
               'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}

# Realizamos la búsqueda con Bayes Search
modeloRL = LogisticRegression()
gs_se = BayesSearchCV(modeloRL, parametros, scoring='roc_auc', verbose=0 , n_jobs=-1, cv = 3, random_state=3)
gs_se.fit(X_train_si_se, y_train_si_se)

# Mostramos los resultados del mejor modelo
print("-"*100)
print("AUC en CV: {:.2f}%".format(gs_se.best_score_ * 100))
print("-"*100)
print("Parámetros del estimador: " + str(gs_se.best_estimator_))
print("-"*100)

# Predecimos los casos de test para ver el accuracy
y_pred_si_se = gs_se.best_estimator_.predict(X_test_si_se)

Es importante notar que en el dataset donde los datos no fueron escalados, si bien la performance del modelo es similar, el mismo codigo nos esta dando una alerta de que no llego a converger y que escalar los valores ayudaria a la convergencia:

```
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
```



Veamos los resultados

In [ ]:
#Exactitud del modelo
print('Exactitud (accuracy) del modelo: {:.2f} %'.format(metrics.roc_auc_score(y_test_si_se, y_pred_si_se)*100))
print("-"*100)

# Reporte del clasificador
from sklearn.metrics import classification_report
print(classification_report(y_test_si_se, y_pred_si_se))

### **CONCLUSIÓN FINAL**

Pudimos ver que no hay gran influencia del **escalado** en el modelo probado, el cual **incrementó solo 1 puntos porcentuales de AUC Score** cuando los datos fueron escalados.
Si bien el escalado no es mandatorio para los modelos de regresion logística a priori, estos ayudan al algoritmo a converger mas rapido asi como tambien agregan "explicabilidad" a los Beta, dado que al estar todas las variables en la misma escala, los betas estan expresados en unidades comparables y eso realmente ayuda a entender que variable explicativa tiene mas o menos peso.

Con Escalado de Variables

In [ ]:
pd.DataFrame({'Atributo':X_train.columns,
              'importancia':abs(gs.best_estimator_.coef_[0])}).sort_values('importancia', ascending=False).head(10)

Sin Escalado de Variables

In [ ]:
pd.DataFrame({'Atributo':X_train.columns,
              'importancia':abs(gs_se.best_estimator_.coef_[0])}).sort_values('importancia', ascending=False).head(10)

Graficamos ambas curvas ROC

In [ ]:
def graficarCurvaRoc( y_pred, model ):
  fpr, tpr, _ = metrics.roc_curve(y_test,  y_pred)
  auc = metrics.roc_auc_score(y_test, y_pred)
  # Graficamos
  plt.plot(fpr,tpr,label= model +" AUC="+str(round(auc,4))) #,label= "AUC="+str(auc))
  plt.legend(loc=4, fontsize=12)
  return auc

# Inicializamos los labels del gráfico
plt.figure(figsize=(20, 10))
plt.xlabel('% Not Hazardous', fontsize=14)
plt.ylabel('% Hazardous', fontsize=14)

# Graficamos la recta del azar
it = [i/100 for i in range(100)]
plt.plot(it,it,label="AZAR AUC=0.5",color="black")

modelos = {'sin-escalar':y_pred_si_se, 'escalando':y_pred_si}
for pred in modelos:
    auc = graficarCurvaRoc( modelos[pred] , pred )

# Agregamos el titulo y configuro el tamaño de letra
plt.title("Curva ROC", fontsize=14)
plt.tick_params(labelsize=12);
plt.show()